In [1]:
%cd ..

/Users/nebula/Documents/MS-DataScience/Fall-2024/DL(DS-677)/Project/TradeForecast


In [2]:
from tradeforecast.augmentation import DataEntryPoint, Indicators, FeatureEngg, RNNDataset

fpath = 'AAPL_1d_max_(None-None).csv'

data_entry = DataEntryPoint(fpath)

indicators = Indicators(data_entry)
indicators.add_macd_sl().add_rsi().add_atr()

features = FeatureEngg(data_entry)
features.add_quarters().add_weeks()

lf = data_entry.data.drop_nulls()

In [ ]:
from torch.utils.data import DataLoader

dataset_kwargs = {'lf': lf,
                 'non_temporal': data_entry.non_temporal,
                 'temporal': data_entry.temporal,
                 'target': data_entry.base_vars,
                 'look_back_len': 30,
                 'forecast_len': 7,
                 'split': 0.2}

train_dataset = RNNDataset(train=True, **dataset_kwargs)
test_dataset = RNNDataset(train=False, **dataset_kwargs)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
from tradeforecast import LSTM
from torch import nn, optim

lstm_kwargs = {'input_size': len(train_dataset.features),
              'hidden_size': 32,
              'n_LSTM': 8,
              'fc_out_size':[64,128,256,128,64],
              'output_size': len(train_dataset.target),
              'forecast_len': train_dataset.forecast_len,
              'dropout': 0.3}

lstm_model = LSTM(**lstm_kwargs)

In [ ]:
lstm_model.train_model(nn.HuberLoss, optim.Adam, 100, train_loader, 0.001)


In [ ]:
y, y_preds = lstm_model.test_model(test_loader)